# RAG with Contextual Retrieval

이번에는 Anthropic의 Contextual Retrieval의 개념을 이용해서 Chunk에 Parent doc을 머지해서 테스트를 진행해봤습니다.


# Configuring Environment

In [1]:
%%capture
!pip install langchain

In [2]:
%%capture
!pip install -U langchain-community

In [3]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain.embeddings.bedrock import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock
from langchain.chains import RetrievalQA
from langchain_community.chat_models import BedrockChat
import boto3

# AWS Bedrock Client Connection with boto3

In [4]:
%%capture
!pip install termcolor

In [5]:
%%capture
!pip install langchain_aws

In [6]:
import json
import boto3
from pprint import pprint
from termcolor import colored

bedrock_client = boto3.client(service_name='bedrock-runtime')

In [7]:
from langchain_aws import ChatBedrock
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler### LLM 로딩 (Claude-v3-sonnet)

from langchain_aws import ChatBedrock
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm_text = ChatBedrock(
    model_id="anthropic.claude-3-5-sonnet-20240620-v1:0",
    client=bedrock_client,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    model_kwargs={
        "max_tokens": 2048,
        "stop_sequences": ["\n\nHuman"],
        # "temperature": 0,
        # "top_k": 350,
        # "top_p": 0.999
    }
)
llm_text

ChatBedrock(callbacks=[<langchain_core.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x7fa25118b250>], client=<botocore.client.BedrockRuntime object at 0x7fa225590a90>, model_id='anthropic.claude-3-5-sonnet-20240620-v1:0', model_kwargs={'stop_sequences': ['\n\nHuman']}, streaming=True, max_tokens=2048)

In [8]:
llm_haiku = ChatBedrock(
    model_id="anthropic.claude-3-haiku-20240307-v1:0",
    client=bedrock_client,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    model_kwargs={
        "max_tokens": 2048,
        "stop_sequences": ["\n\nHuman"],
        # "temperature": 0,
        # "top_k": 350,
        # "top_p": 0.999
    }
)
llm_haiku

ChatBedrock(callbacks=[<langchain_core.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x7fa224df0340>], client=<botocore.client.BedrockRuntime object at 0x7fa225590a90>, model_id='anthropic.claude-3-haiku-20240307-v1:0', model_kwargs={'stop_sequences': ['\n\nHuman']}, streaming=True, max_tokens=2048)

In [9]:
llm_haiku.invoke(input="What is the recipe of mayonnaise?")

AIMessage(content='The basic recipe for mayonnaise is:\n\nIngredients:\n- 1 egg yolk\n- 1 tsp Dijon mustard\n- 1 tbsp lemon juice or white wine vinegar\n- 1 cup vegetable oil (such as canola or grapeseed oil)\n- Salt and pepper to taste\n\nInstructions:\n1. In a medium bowl, whisk together the egg yolk, mustard, and lemon juice/vinegar until well combined.\n2. Very slowly, while whisking constantly, drizzle in the vegetable oil a few drops at a time. As the emulsion forms, you can start adding the oil in a thin stream, continuing to whisk vigorously.\n3. Once all the oil is incorporated, the mayonnaise should be thick and creamy. Season with salt and pepper to taste.\n4. Store in the refrigerator for up to 1 week.\n\nThe key techniques are:\n- Using a whole egg yolk, not just the whites\n- Adding the oil very slowly at first to help the emulsion form\n- Whisking constantly to incorporate the oil fully\n\nThis produces a classic, creamy homemade mayonnaise. You can adjust the flavors by

In [10]:
from langchain.embeddings import BedrockEmbeddings

llm_emb = BedrockEmbeddings(
    client=bedrock_client,
    model_id="Titan-Embeddings-G1"
)
dimension = 1536
print("Bedrock Embeddings Model Loaded")

Bedrock Embeddings Model Loaded


/tmp/ipykernel_14356/3925743753.py:3: LangChainDeprecationWarning: The class `BedrockEmbeddings` was deprecated in LangChain 0.2.11 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-aws package and should be used instead. To use it run `pip install -U :class:`~langchain-aws` and import as `from :class:`~langchain_aws import BedrockEmbeddings``.
  llm_emb = BedrockEmbeddings(


# PDF Files Loading with PyMuPDF

In [11]:
%%capture
!pip install pymupdf4llm

### PDF를 파싱해서 Markdown file로 변환합니다.

In [43]:
!wget https://byekang-share-materials.s3.ap-northeast-2.amazonaws.com/file-public-share/20120401_LIG_House_Fire_Insurance_Pol_Condition.pdf -O 20120401_LIG_House_Fire_Insurance_Pol_Condition.pdf
!wget https://byekang-share-materials.s3.ap-northeast-2.amazonaws.com/file-public-share/20120401_LIG_House_Fire_Insurance_Pol_Condition_1-4.pdf -O 20120401_LIG_House_Fire_Insurance_Pol_Condition_1-4.pdf

--2024-11-10 19:54:57--  https://byekang-share-materials.s3.ap-northeast-2.amazonaws.com/file-public-share/20120401_LIG_House_Fire_Insurance_Pol_Condition.pdf
Resolving byekang-share-materials.s3.ap-northeast-2.amazonaws.com (byekang-share-materials.s3.ap-northeast-2.amazonaws.com)... 3.5.186.151, 3.5.140.189, 3.5.144.243, ...
Connecting to byekang-share-materials.s3.ap-northeast-2.amazonaws.com (byekang-share-materials.s3.ap-northeast-2.amazonaws.com)|3.5.186.151|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 426490 (416K) [application/pdf]
Saving to: ‘20120401_LIG_House_Fire_Insurance_Pol_Condition.pdf’

100%[======================================>] 426,490      478KB/s   in 0.9s   

2024-11-10 19:54:59 (478 KB/s) - ‘20120401_LIG_House_Fire_Insurance_Pol_Condition.pdf’ saved [426490/426490]

--2024-11-10 19:54:59--  https://byekang-share-materials.s3.ap-northeast-2.amazonaws.com/file-public-share/20120401_LIG_House_Fire_Insurance_Pol_Condition_1-4.pdf
Resol

In [12]:
import pymupdf4llm
md_text = pymupdf4llm.to_markdown("20120401_LIG_House_Fire_Insurance_Pol_Condition_1-4.pdf")

### Markdown file로 잘 변화되었는지를 확인하기 위해서 화면으로 display해 봅니다.

In [13]:
import re
md_text = re.sub('LIG주택화재보험 보통약관', '# LIG주택화재보험 보통약관', md_text)
md_text = re.sub(r'\n제([0-9]*)조\((.*)\)\n', r'\n# 제\1조(\2)\n', md_text)
md_text = re.sub(r'\n제([0-9]*)조\[(.*)\]\n', r'\n# 제\1조[\2]\n', md_text)
md_text = re.sub(r'\n([①②③④⑤])', r'\n## \1\n', md_text)

In [14]:
import pathlib
pathlib.Path("output.md").write_bytes(md_text.encode())

19433

In [15]:
from IPython.display import Markdown, display

display(Markdown(md_text))

# LIG주택화재보험 보통약관

# 제1조(보험계약의 성립)
## ①
 보험계약은 보험계약자의 청약과 보험회사의 승낙으로 이루어집니다.(이하 “ 보험계약”은 “ 계약”, “ 보

험계약자” 는 “ 계약자” , “ 보험회사” 는 “ 회사” 라 합니다)

## ②
 회사는 계약의 청약을 받고 보험료 전액(보험료를 일시에 지급하기로 약정한 경우) 또는 제1회 보험

료(보험료를 분납하기로 약정한 경우)(이하 ‘ 제1회 보험료 등’ 이라 합니다)를 받은 경우에는 청약일
부터 30일 이내에 승낙 또는 거절의 통지를 하며 통지가 없으면 승낙한 것으로 봅니다.

## ③
 회사가 청약을 승낙한 때에는 지체 없이 보험증권(보험가입증서)을 계약자에게 교부하여 드리며, 청

약을 거절한 경우에는 거절통지와 함께 받은 금액을 계약자에게 돌려드립니다.

## ④
 이미 성립한 계약을 연장하거나 변경하는 경우에는 회사는 보험증권(보험가입증서)에 그 사실을 기

재함으로써 보험증권(보험가입증서)의 교부에 대신할 수 있습니다.

# 제2조(약관교부 및 설명의무 등)
## ①
 회사는 계약자가 청약한 경우 계약자에게 약관 및 계약자 보관용 청약서(청약서 부본)를 드리고 약

관의 중요한 내용을 설명하여 드립니다. 다만, 전화․ 우편․ 컴퓨터 등의 통신매체를 통한 보험계약
(이하 “ 통신판매 계약” 이라 합니다)의 경우에는 회사는 계약자의 동의를 얻어 다음 중 한 가지 방
법으로 할 수 있습니다.

1. 광기록매체 또는 전자우편 등의 전자적 방법으로 약관이나 계약자 보관용 청약서(청약서 부본)를

송부하는 방법. 이 경우 계약자 또는 그 대리인이 이를 수신한 때에 당해 약관이나 계약자 보관용
청약서(청약서 부본)를 드린 것으로 봅니다.

2. 사이버몰(컴퓨터를 이용하여 보험거래를 할 수 있도록 설정된 가상의 영업장)에서 약관 및 그 설명

문(약관의 중요한 내용을 알 수 있도록 설명한 문서)을 읽거나 내려받게 하는 방법. 이 경우 계약
자가 이를 읽거나 내려받은 것을 확인한 때에 당해 약관을 드리고 그 중요한 내용을 설명한 것으
로 봅니다.

3. 전화를 이용하여 청약내용, 보험료납입, 보험기간, 계약전 알릴의무, 약관의 중요한 내용 등 계약

체결을 위하여 필요한 사항을 질문 또는 설명하는 방법. 이 경우 계약자의 답변과 확인내용을 음성
녹음함으로써 약관의 중요한 내용을 설명한 것으로 봅니다.

## ②
 회사가 제1항에 의해 제공될 약관 및 계약자 보관용 청약서(청약서 부본)를 청약 시 계약자에게 드

리지 아니하거나 약관의 중요한 내용을 설명하지 아니한 때 또는 계약을 체결할 때 계약자가 청약서
에 자필서명[날인(도장을 찍음) 또는 전자서명법 제2조 제10호의 규정에 의한 공인인증기관이 인증
한 전자서명을 포함합니다)]을 하지 아니한 때에는 계약자는 계약체결일부터 1개월 이내에 계약을
취소할 수 있습니다.

## ③
 제2항에 따라 계약이 취소된 경우에는 회사는 이미 납입한 보험료를 계약자에게 돌려드리며, 보험료

를 받은 기간에 대하여 보험개발원이 공시하는 정기예금이율을 연단위 복리로 계산한 금액을 더하여
지급합니다.

# 제3조(제1회 보험료 등 및 회사의 보장개시)
## ①
 회사는 계약의 청약을 승낙하고 제1회 보험료 등을 받은 때부터 이 약관이 정한 바에 따라 보장을

합니다.

## ②
 회사가 계약자로부터 계약의 청약과 함께 제1회 보험료 등을 받은 경우에 그 청약을 승낙하기 전에

계약에서 정한 보험금 지급사유가 생긴 때에는 회사는 계약상의 보장을 합니다.

## ③
 제2항의 규정에도 불구하고 회사는 다음 중 한 가지에 해당되는 경우에는 보장을 하지 아니합니다.

1. 제14조(계약전 알릴 의무)의 규정에 의하여 계약자 또는 피보험자가 회사에 알린 내용이 보험금

지급사유의 발생에 영향을 미쳤음을 회사가 증명하는 경우

2. 제12조(보상하지 아니하는 손해), 제16조(계약의 해지), 제18조(계약의 무효) 또는 제27조(사기에

의한 계약)의 규정을 준용하여 회사가 보장을 하지 아니할 수 있는 경우

## ④
 계약자가 제1회 보험료 등을 자동이체 또는 신용카드로 납입하는 경우에는 자동이체신청 및 신용카

드매출 승인에 필요한 정보를 회사에 제공한 때가 제1회 보험료 등을 납입한 때가 되나, 계약자의
귀책사유로 자동이체 또는 매출승인이 불가능한 경우에는 제1회 보험료 등이 납입되지 않은 것으로
봅니다.


-----

## ⑤
 계약이 갱신되는 경우에는 제1항 내지 제3항에 의한 보장은 기존 계약에 의한 보장이 종료하는 때부

터 적용합니다.

# 제4조(제2회 이후 보험료의 납입)
계약자는 제2회 이후의 보험료를 계약 체결시 납입하기로 약속한 날(이하฀납입기일฀이라 합니다)까
지 납입하여야 하며, 회사는 계약자가 보험료를 납입한 경우에는 영수증을 발행하여 드립니다. 다만, 금
융회사(우체국을 포함합니다)룰 통하여 보험료를 납입한 경우에는 그 금융회사 발행 증빙서류를 영수증
으로 대신합니다.

# 제5조(청약의 철회)
## ①
 가계성 보험[개인의 일상생활과 관련된 보험으로 보험료를 단체 또는 법인이 부담하지 않는 개인보

험계약과 단체요율(피보험자가 단체인 계약에 적용되는 요율)이 적용되지 않는 계약을 말합니다]에
한하여 계약자는 청약을 한 날부터 15일 이내에 그 청약을 철회할 수 있습니다. 다만, 단체(취급)계
약 또는 보험기간이 1년 미만인 계약의 경우에는 청약을 철회할 수 없으며, 통신판매 계약(보험기간
이 1년 이상인 계약에 한함)의 경우에는 청약을 한 날부터 30일 이내에 그 청약을 철회할 수 있습니
다.

## ②
 회사는 청약의 철회를 접수한 경우에는 3일 이내에 이미 납입한 보험료를 계약자에게 돌려 드리며,

보험료 반환이 지체된 기간에 대하여는 보험개발원이 공시하는 정기예금이율을 연단위 복리로 계산
한 금액을 더하여 지급합니다. 다만, 계약자가 제1회 보험료 등을 신용카드로 납입한 계약의 청약을
철회하는 경우에 회사는 신용카드의 매출을 취소하며 이자를 더하여 지급하지 아니합니다.

## ③
 청약을 철회할 당시에 이미 보험금 지급사유가 발생하였으나 계약자가 그 보험금 지급사유의 발생사

실을 알지 못한 경우에는 청약철회의 효력은 발생하지 않습니다.

# 제6조(계약내용의 변경 등)
## ①
 계약자는 회사의 승낙을 얻어 다음의 사항을 변경할 수 있습니다. 이 경우 승낙을 서면 등으로 알리

거나 보험증권(보험가입증서)의 뒷면에 기재하여 드립니다.

1. 보험종목
2. 보험기간
3. 보험료 납입주기, 수금방법 및 납입기간
4. 계약자, 피보험자, 보험가입금액 등 기타 계약의 내용

## ②
 회사는 계약자가 제1회 보험료 등을 납입한 때부터 1년 이상 경과된 유효한 계약으로서 그 보험종목

의 변경을 요청할 때에는 회사의 사업방법서에서 정하는 방법에 따라 이를 변경하여 드립니다.

## ③
 회사는 계약자가 제1항 제4호의 규정에 의하여 보험가입금액을 감액하고자 할 때에는 그 감액된 부

분은 계약이 해지된 것으로 보며, 이로 인하여 회사가 환급하여야 할 보험료가 있을 경우에는 19조
(보험료의 환급)에 따라 이를 계약자에게 지급합니다.

# 제7조[보험료의 납입연체시 납입최고(독촉)와 계약의 해지]
## ①
 계약자가 제2회 이후의 보험료를 납입기일까지 납입하지 아니하여 보험료 납입이 연체 중인 경우에

는, 회사는 14일(보험기간이 1년 미만인 경우에는 7일) 이상의 기간을 납입최고(독촉)기간으로 정하
여 계약자(타인을 위한 계약의 경우 그 특정된 타인을 포함합니다)에게 다음의 내용을 서면(등기우편
등), 전화(음성녹음) 또는 전자문서 등으로 알려드립니다. 다만, 계약이 해지되기 전에 발생한 보험금
지급사유에 대하여 회사는 계약상의 보장을 합니다.

1. 납입최고(독촉)기간 안에 연체보험료를 납입하여야 한다는 내용
2. 납입최고(독촉)기간이 끝나는 날까지 보험료를 납입하지 아니할 경우 그 끝나는 날의 다음날에 계

약이 해지된다는 내용

3. 계약자가 회사로부터 보험계약대출을 받은 경우에는, 계약이 해지되는 즉시 해지환급금에서 보험계

약대출 원리금이 차감된다는 내용

## ②
 제1항의 납입최고(독촉)기간은 납입최고(독촉)의 통지가 계약자(타인을 위한 계약의 경우에는 그 특

정된 타인을 포함)에게 도달한 날부터 시작되며, 그 기간의 말일(말일이 토요일 또는 공휴일인 경우
에는 그 다음의 최초의 평일)에 끝납니다.

## ③
 회사가 제1항에 의한 납입최고(독촉) 등을 전자문서로 안내하고자 할 경우에는 계약자의 서면에 의

한 동의를 얻어 수신확인을 조건으로 전자문서를 송신하여야 하며, 계약자가 전자문서에 대하여 수
신을 확인하기 전까지는 그 전자문서는 송신되지 아니한 것으로 봅니다. 회사는 전자문서가 수신되


-----

지 아니한 것으로 확인되는 경우에는 제1항의 납입최고(독촉)기간을 설정하여 제1항에서 정한 내용
을 서면(등기우편 등) 또는 전화(음성녹음)로 다시 알려 드립니다.

## ④
 제1항에 따라 계약이 해지된 경우에는 이로 인하여 회사가 환급하여야 할 보험료가 있을 경우에는

제19조(보험료의 환급)에 따라 이를 계약자에게 지급합니다.

# 제8조[보험료의 납입연체로 인한 해지계약의 부활(효력회복)]
## ①
 제7조[보험료의 납입연체시 납입최고(독촉)와 계약의 해지]에 따라 계약이 해지되었으나 계약자가

제19조(보험료의 환급)에 따른 환급금을 받지 아니한 경우 계약자는 해지된 날부터 2년 이내에 회사
가 정한 절차에 따라 계약의 부활(효력회복)을 청약할 수 있습니다. 이 경우 회사가 그 청약을 승낙
한 때에는 계약자는 부활(효력회복)을 청약한 날까지의 연체된 보험료에 보험개발원이 공시하는 정
기예금이율 + 1% 범위내에서 각 상품별로 회사가 정하는 이율로 계산한 금액을 더하여 납입하여야
합니다.

## ②
 제1항에 따라 해지계약을 부활(효력회복)하는 경우에는 제1조(보험계약의 성립), 제3조(제1회 보험료

등 및 회사의 보장개시), 제14조(계약 전 알릴의무) 및 제16조(계약의 해지)의 규정을 준용합니다.

# 제9조[강제집행 등으로 인한 해지계약의 특별부활(효력회복)]
## ①
 타인을 위한 계약의 경우 제19조(보험료의 환급)에 따른 계약자의 해지환급금 청구권에 대한 강제집

행, 담보권실행, 국세 및 지방세 체납처분절차에 의해 계약이 해지된 경우에는, 회사는 해지 당시의
피보험자가 계약자의 동의를 얻어 계약 해지로 회사가 채권자에게 지급한 금액을 회사에게 지급하고
제6조(계약내용의 변경 등) 제1항의 절차에 따라 계약자 명의를 피보험자로 변경하여 계약의 특별부
활(효력회복)을 청약할 수 있음을 피보험자에게 통지하여야 합니다.

## ②
 회사는 제1항에 의한 계약자 명의변경 신청 및 계약의 특별부활(효력회복) 청약을 승낙하며, 계약은

청약한 때부터 특별부활(효력회복) 됩니다.

## ③
 회사는 제1항의 통지를 계약이 해지된 날부터 7일 이내에 하여야 합니다. 다만, 회사의 통지가 7일

을 경과하여 도달하고 이후 피보험자가 제1항에 의한 계약자 명의변경 신청 및 계약의 특별부활(효
력회복)을 청약한 경우에는 계약이 해지된 날부터 7일이 되는 날에 특별부활(효력회복) 됩니다.

## ④
 피보험자는 통지를 받은 날부터 15일 이내에 제1항의 절차를 이행할 수 있습니다.

# 제10조(보험목적의 범위)
## ①
이 약관에서 보험의 목적이라 함은 이 약관에 따라 보험에 가입한 물건으로서 보험증권(보험가입증

서)에 기재된 건물 등을 말합니다.

1. 주택으로만 쓰이는 건물 중 다음의 것 또는 그 수용가재

(ⅰ) 단독주택
(ⅱ) 주택의 부속건물로서 가재만을 수용하는데 쓰이는 것
(ⅲ) 연립(다세대)주택, 아파트로서 각 호(戶), 실(室)이 모두 주택으로만 쓰이는 것

2. 주택병용 물건으로서 아래의 용도로 사용하는 건물 및 그 수용가재

(ⅰ) 교습소(피아노, 꽃꽂이, 국악, 재봉 및 이와 비슷한 것)
(ⅱ) 치료(안수, 침구(침질,뜸질), 정골, 조산원 및 이와 비슷한 것)

## ②
 아래의 물건은 보험증권(보험가입증서)에 기재하여야만 보험의 목적이 됩니다.

1. 통화, 유가증권, 인지, 우표 및 이와 비슷한 것
2. 귀금속, 귀중품(무게나 부피가 휴대할 수 있으며 점당 300만원 이상), 보옥, 보석, 글․ 그림, 골동품,

조각물 및 이와 비슷한 것

3. 원고, 설계서, 도안, 물건의 원본, 모형, 증서, 장부, 금형(쇠틀), 목형(나무틀), 소프트웨어 및 이와

비슷한 것

## ③
 아래의 물건은 다른 약정이 없으면 보험의 목적에 포함됩니다.

1. 건물인 경우
(ⅰ)건물의 부속물: 피보험자의 소유인 칸막이, 대문, 담, 곳간 및 이와 비슷한 것
(ⅱ)건물의 부착물: 피보험자 소유인 간판, 네온사인, 안테나, 선전탑 및 이와 비슷한 것
2. 가재인 경우: 피보험자와 같은 세대에 속하는 사람의 소유물

# 제11조(보상하는 손해)
## ①
회사는 보험의 목적이 화재(벼락을 포함합니다), 폭발 또는 파열(이하「사고」라 합니다)로 입은 아래


-----

의 손해를 보상하여 드립니다.

1. 사고에 따른 직접손해
2. 사고에 따른 소방손해(화재진압과정에서 발생하는 손해)
3. 사고에 따른 피난손해(피난지에서 5일 동안에 보험의 목적에 생긴 위 제1호 및 제2호의 손해를 포

함합니다)

## ②
 회사는 제1항에서 보장하는 위험으로 인하여 손해가 발생한 경우 계약자 또는 피보험자가 지출한 아

래의 비용을 추가로 지급합니다.

1. 잔존물 제거비용: 사고현장에서의 잔존물의 해체비용, 청소비용(사고현장 및 인근 지역의 토양, 대

기 및 수질 오염물질 제거비용과 차에 실은 후 폐기물 처리비용은 포함되지 아니합니다) 및 차에
싣는 비용. 다만, 제1항에서 보장하지 아니하는 위험으로 보험의 목적이 손해를 입거나 관계법령에
의하여 제거됨으로써 생긴 손해에 대하여는 보상하여 드리지 아니합니다.

2. 손해방지비용: 손해의 방지 또는 경감을 위하여 지출한 필요 또는 유익한 비용
3. 대위권 보전비용: 제3자로부터 손해의 배상을 받을 수 있는 경우에는 그 권리를 지키거나 행사하기

위하여 지출한 필요 또는 유익한 비용

4. 잔존물 보전비용: 잔존물을 보전하기 위하여 지출한 필요 또는 유익한 비용. 다만, 제30조(잔존물)

에 의해 회사가 잔존물을 취득한 경우에 한합니다.

5. 기타 협력비용: 회사의 요구에 따르기 위하여 지출한 필요 또는 유익한 비용

# 제12조(보상하지 아니하는 손해)
회사는 아래의 사유로 인한 손해는 보상하여 드리지 아니합니다.

1. 계약자, 피보험자(법인인 경우에는 그 이사 또는 법인의 업무를 집행하는 그 밖의 기관) 또는 이들

의 법정대리인의 고의 또는 중대한 과실

2. 화재, 폭발 또는 파열이 발생했을 때의 분실로 생긴 손해
3. 보험의 목적의 발효, 자연발열, 자연발화로 생긴 손해. 그러나, 자연발열 또는 자연발화로 연소된

다른 보험의 목적에 생긴 손해는 보상하여 드립니다.

4. 화재, 폭발 또는 파열로 인하지 아니한 수도관, 수관, 수압기 등의 파열로 생긴 손해
5. 발전기, 여자기(정류기 포함), 변류기, 변압기, 전압조정기, 축전기, 개폐기, 차단기, 피뢰기, 배전반

및 그 밖의 전기기기 또는 장치의 전기적 사고로 생긴 손해. 그러나 그 결과로 생긴 화재, 폭발, 파
열손해는 보상하여 드립니다.

6. 원인의 직접, 간접을 묻지 아니하고 지진, 분화 또는 전쟁, 혁명, 내란, 사변, 폭동, 소요, 노동쟁의,

기타 이들과 유사한 사태로 생긴 화재 및 연소 또는 그 밖의 손해

7. 핵연료물질 (사용된 연료를 포함합니다. 이하 같습니다) 또는 핵연료 물질에 의하여 오염된 물질

(원자핵 분열 생성물을 포함합니다)의 방사성, 폭발성 그 밖의 유해한 특성 또는 이들의 특성에 의
한 사고로 인한 손해

8. 위 제7호 이외의 방사선을 쬐는 것 또는 방사능 오염으로 인한 손해
9. 국가 및 지방자치단체의 명령에 의한 재산의 소각 및 이와 유사한 손해

# 제13조(보험금 등의 지급한도)
## ①
 제11조(보상하는 손해)제1항의 손해에 의한 보험금과 제11조(보상하는 손해)제2항의 잔존물 제거비

용은 각각 제23조(지급보험금의 계산)를 준용하여 계산하며, 그 합계액은 보험증권(보험가입증서)에
기재된 보험가입금액을 한도로 합니다. 다만, 잔존물 제거비용은 손해액의 10%를 초과할 수 없습니
다.

## ②
 제11조(보상하는 손해)제2항의 비용손해 중 손해방지비용, 대위권 보전비용 및 잔존물 보전비용은

제23조(지급보험금의 계산)를 준용하여 계산한 금액이 보험가입금액을 초과하는 경우에도 이를 지급
합니다.

## ③
 제11조(보상하는 손해)제2항의 비용손해 중 기타 협력비용은 보험가입금액을 초과한 경우에도 이를

전액 지급합니다.

# 제14조(계약전 알릴 의무)
계약자, 피보험자 또는 이들의 대리인은 청약시 청약서(질문서를 포함합니다)에서 질문한 사항에 대하
여 알고 있는 사실을 반드시 사실대로 알려야 합니다.

<용어풀이>


-----



# MarkDown 파일을 목차별로 split합니다.

https://python.langchain.com/docs/how_to/markdown_header_metadata_splitter/

In [16]:
from langchain_text_splitters import MarkdownHeaderTextSplitter

In [17]:
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

In [18]:
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on)
md_header_splits = markdown_splitter.split_text(md_text)

In [19]:
md_header_splits

[Document(metadata={'Header 1': '제1조(보험계약의 성립)', 'Header 2': '①'}, page_content='보험계약은 보험계약자의 청약과 보험회사의 승낙으로 이루어집니다.(이하 “ 보험계약”은 “ 계약”, “ 보  \n험계약자” 는 “ 계약자” , “ 보험회사” 는 “ 회사” 라 합니다)'),
 Document(metadata={'Header 1': '제1조(보험계약의 성립)', 'Header 2': '②'}, page_content='회사는 계약의 청약을 받고 보험료 전액(보험료를 일시에 지급하기로 약정한 경우) 또는 제1회 보험  \n료(보험료를 분납하기로 약정한 경우)(이하 ‘ 제1회 보험료 등’ 이라 합니다)를 받은 경우에는 청약일\n부터 30일 이내에 승낙 또는 거절의 통지를 하며 통지가 없으면 승낙한 것으로 봅니다.'),
 Document(metadata={'Header 1': '제1조(보험계약의 성립)', 'Header 2': '③'}, page_content='회사가 청약을 승낙한 때에는 지체 없이 보험증권(보험가입증서)을 계약자에게 교부하여 드리며, 청  \n약을 거절한 경우에는 거절통지와 함께 받은 금액을 계약자에게 돌려드립니다.'),
 Document(metadata={'Header 1': '제1조(보험계약의 성립)', 'Header 2': '④'}, page_content='이미 성립한 계약을 연장하거나 변경하는 경우에는 회사는 보험증권(보험가입증서)에 그 사실을 기  \n재함으로써 보험증권(보험가입증서)의 교부에 대신할 수 있습니다.'),
 Document(metadata={'Header 1': '제2조(약관교부 및 설명의무 등)', 'Header 2': '①'}, page_content='회사는 계약자가 청약한 경우 계약자에게 약관 및 계약자 보관용 청약서(청약서 부본)를 드리고 약  \n관의 중요한 내용을 설명하여 드립니다. 다만, 전화․ 우편․ 컴퓨터 등의 통신매체를 통한 보험계약\n(이하 “ 

### 몇개의 docuement로 split되었는지 출력해 봅니다.

In [20]:
for index, doc in enumerate(md_header_splits):
    print("[Header Doc Index]", index, "------------------------")
    print("  Header 1: ", doc.metadata['Header 1'])
    
    if 'Header 2' in doc.metadata :
        print("    Header 2: ", doc.metadata['Header 2'])
    
    print("      [Doc page_content]:", doc.page_content)
    print("")
    print("")

[Header Doc Index] 0 ------------------------
  Header 1:  제1조(보험계약의 성립)
    Header 2:  ①
      [Doc page_content]: 보험계약은 보험계약자의 청약과 보험회사의 승낙으로 이루어집니다.(이하 “ 보험계약”은 “ 계약”, “ 보  
험계약자” 는 “ 계약자” , “ 보험회사” 는 “ 회사” 라 합니다)


[Header Doc Index] 1 ------------------------
  Header 1:  제1조(보험계약의 성립)
    Header 2:  ②
      [Doc page_content]: 회사는 계약의 청약을 받고 보험료 전액(보험료를 일시에 지급하기로 약정한 경우) 또는 제1회 보험  
료(보험료를 분납하기로 약정한 경우)(이하 ‘ 제1회 보험료 등’ 이라 합니다)를 받은 경우에는 청약일
부터 30일 이내에 승낙 또는 거절의 통지를 하며 통지가 없으면 승낙한 것으로 봅니다.


[Header Doc Index] 2 ------------------------
  Header 1:  제1조(보험계약의 성립)
    Header 2:  ③
      [Doc page_content]: 회사가 청약을 승낙한 때에는 지체 없이 보험증권(보험가입증서)을 계약자에게 교부하여 드리며, 청  
약을 거절한 경우에는 거절통지와 함께 받은 금액을 계약자에게 돌려드립니다.


[Header Doc Index] 3 ------------------------
  Header 1:  제1조(보험계약의 성립)
    Header 2:  ④
      [Doc page_content]: 이미 성립한 계약을 연장하거나 변경하는 경우에는 회사는 보험증권(보험가입증서)에 그 사실을 기  
재함으로써 보험증권(보험가입증서)의 교부에 대신할 수 있습니다.


[Header Doc Index] 4 ------------------------
  Header 1:  제2조(약관교부 및 설명의무

# Child Chunk 생성
### RecursiveCharacterTextSplitter를 이용해서 RAG로 사용할 수 있도록 child chunk로 생성합니다.

In [21]:
# Char-level splits
from langchain_text_splitters import RecursiveCharacterTextSplitter

chunk_size = 250
chunk_overlap = 30
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

## Context 생성용 모델 세팅

In [22]:
WHOLE_DOCUMENT = """Amazon Elastic Compute Cloud(Amazon EC2)는 Amazon Web Services(AWS) 클라우드에서 온디맨
드 확장 가능 컴퓨팅 용량을 제공합니다. Amazon EC2를 사용하면 하드웨어 비용이 절감되므로 애플
리케이션을 더욱 빠르게 개발하고 배포할 수 있습니다. Amazon EC2를 사용하여 원하는 수의 가상 서
버를 구축하고 보안 및 네트워킹을 구성하며 스토리지를 관리할 수 있습니다. 용량을 추가(스케일 업)
하여 월간 또는 연간 프로세스 또는 웹 사이트 트래픽 급증 등 컴퓨팅 사용량이 많은 작업을 처리할 수
있습니다. 사용량이 감소하면 용량을 다시 축소(스케일 다운)할 수 있습니다.  
EC2 인스턴스는 AWS 클라우드의 가상 서버입니다. EC2 인스턴스를 시작할 때 지정하는 인스턴스 유
형에 따라 인스턴스에 사용할 수 있는 하드웨어가 결정됩니다. 인스턴스 유형마다 서로 다른 컴퓨팅,
메모리, 네트워크 및 스토리지 리소스의 균형을 제공합니다. 자세한 내용은 Amazon EC2 인스턴스 유
형 안내서 를 참조하세요."""

In [23]:
CHUNK_CONTENT = """Amazon Elastic Compute Cloud(Amazon EC2)는 Amazon Web Services(AWS) 클라우드에서 온디맨
드 확장 가능 컴퓨팅 용량을 제공합니다. Amazon EC2를 사용하면 하드웨어 비용이 절감되므로 애플
리케이션을 더욱 빠르게 개발하고 배포할 수 있습니다. Amazon EC2를 사용하여 원하는 수의 가상 서
버를 구축하고 보안 및 네트워킹을 구성하며 스토리지를 관리할 수 있습니다. 용량을 추가(스케일 업)"""

In [24]:
CONTEXUALIZER_PROMPT_TEMPLATE = f"""\
<document> 
{WHOLE_DOCUMENT} 
</document> 
Here is the chunk we want to situate within the whole document 
<chunk> 
{CHUNK_CONTENT} 
</chunk> 
Please give a short succinct context to situate this chunk within the overall document for the purposes of improving search retrieval of the chunk. Answer only with the succinct context and nothing else. Answer in Korean. 
"""

In [25]:
llm_response=llm_haiku.invoke(input=CONTEXUALIZER_PROMPT_TEMPLATE)

In [26]:
print(llm_response.content)

본 문서는 Amazon Elastic Compute Cloud(Amazon EC2)에 대한 설명을 제공하는 문서입니다. 제시된 chunk는 Amazon EC2가 온디맨드 확장 가능 컴퓨팅 용량을 제공하고 하드웨어 비용을 절감하여 애플리케이션 개발과 배포를 용이하게 하며, 가상 서버 구축, 보안 및 네트워킹 구성, 스토리지 관리 등의 기능을 제공한다는 내용을 설명하고 있습니다.


## Contextual Chunk를 생성합니다.
목차별로 나눠진 chunk (parent chunk)의 맨 뒷부분에 child chunk를 머지해서 Contextual Chunk를 생성합니다.
그리고 나서 해당 Chunk를 page_content로 업데이트해서 저장합니다.

In [27]:
splits = []

for index, parent_doc in enumerate(md_header_splits):
    #print("[Parent Doc Index]", index, "------------------------")
    #print("--> parent_doc.metadata: ", parent_doc.metadata)
    parent_page_content = 'Header of this content is ' + parent_doc.metadata['Header 1'] + '. ' + parent_doc.page_content
    #print("--> parent_page_content:", parent_page_content)
    
    child_docs = text_splitter.split_documents([parent_doc])
    
    for child_doc in child_docs:
        original_child_chunk = child_doc.page_content
        CHUNK_CONTENT = original_child_chunk
        WHOLE_DOCUMENT = parent_page_content
        CONTEXUALIZER_PROMPT_TEMPLATE = f"""\
<document> 
{WHOLE_DOCUMENT} 
</document> 
Here is the chunk we want to situate within the whole document 
<chunk> 
{CHUNK_CONTENT} 
</chunk> 
Please give a short succinct context to situate this chunk within the overall document for the purposes of improving search retrieval of the chunk. 
Mention the title of the header that this chunk belongs to, followed by a short succinct context.
Answer in Korean. 
"""
        llm_reponse=llm_haiku.invoke(input=CONTEXUALIZER_PROMPT_TEMPLATE)
        contexual_child_chunk = llm_reponse.content + ' !! ' + original_child_chunk
        child_doc.page_content = contexual_child_chunk
        print("----> contexual_child_chunk: ", contexual_child_chunk)
        
    splits = splits + child_docs

----> contexual_child_chunk:  <document_context>
제1조(보험계약의 성립): 이 문단은 보험계약의 성립 과정을 설명하고 있습니다. 보험계약은 보험계약자의 청약과 보험회사의 승낙으로 이루어지며, 이하 "보험계약"은 "계약", "보험계약자"는 "계약자", "보험회사"는 "회사"로 표현됩니다.
</document_context> !! 보험계약은 보험계약자의 청약과 보험회사의 승낙으로 이루어집니다.(이하 “ 보험계약”은 “ 계약”, “ 보  
험계약자” 는 “ 계약자” , “ 보험회사” 는 “ 회사” 라 합니다)
----> contexual_child_chunk:  이 내용은 <document>에서 '제1조(보험계약의 성립)'이라는 제목의 헤더 부분에 해당합니다. 이 부분은 보험회사가 계약 청약을 받고 보험료를 받은 경우, 청약일로부터 30일 이내에 승낙 또는 거절 통지를 해야 한다는 내용을 설명하고 있습니다. !! 회사는 계약의 청약을 받고 보험료 전액(보험료를 일시에 지급하기로 약정한 경우) 또는 제1회 보험  
료(보험료를 분납하기로 약정한 경우)(이하 ‘ 제1회 보험료 등’ 이라 합니다)를 받은 경우에는 청약일
부터 30일 이내에 승낙 또는 거절의 통지를 하며 통지가 없으면 승낙한 것으로 봅니다.
----> contexual_child_chunk:  이 내용은 <제1조(보험계약의 성립)>의 일부로, 보험회사가 고객의 청약을 승낙하거나 거절할 경우 어떤 절차를 거치는지 설명하고 있습니다. 즉, 보험회사가 청약을 승낙하면 지체없이 보험증권을 발급해 주어야 하며, 청약을 거절할 경우에는 거절통지와 함께 받은 금액을 돌려주어야 한다는 내용입니다. !! 회사가 청약을 승낙한 때에는 지체 없이 보험증권(보험가입증서)을 계약자에게 교부하여 드리며, 청  
약을 거절한 경우에는 거절통지와 함께 받은 금액을 계약자에게 돌려드립니다.
----> contexual_child_chunk:  <document-context>
제1조(보험계약의 성

In [28]:
#splits

In [29]:
### 몇개의 청크로 생성되었는지 출력해 봅니다.

In [30]:
for index, doc in enumerate(splits):
    print("[Header Doc Index]", index, "------------------------")
    print("  Header 1: ", doc.metadata['Header 1'])
    
    if 'Header 2' in doc.metadata :
        print("    Header 2: ", doc.metadata['Header 2'])
    
    print("      [Doc page_content]:", doc.page_content)
    print("")
    print("")

[Header Doc Index] 0 ------------------------
  Header 1:  제1조(보험계약의 성립)
    Header 2:  ①
      [Doc page_content]: <document_context>
제1조(보험계약의 성립): 이 문단은 보험계약의 성립 과정을 설명하고 있습니다. 보험계약은 보험계약자의 청약과 보험회사의 승낙으로 이루어지며, 이하 "보험계약"은 "계약", "보험계약자"는 "계약자", "보험회사"는 "회사"로 표현됩니다.
</document_context> !! 보험계약은 보험계약자의 청약과 보험회사의 승낙으로 이루어집니다.(이하 “ 보험계약”은 “ 계약”, “ 보  
험계약자” 는 “ 계약자” , “ 보험회사” 는 “ 회사” 라 합니다)


[Header Doc Index] 1 ------------------------
  Header 1:  제1조(보험계약의 성립)
    Header 2:  ②
      [Doc page_content]: 이 내용은 <document>에서 '제1조(보험계약의 성립)'이라는 제목의 헤더 부분에 해당합니다. 이 부분은 보험회사가 계약 청약을 받고 보험료를 받은 경우, 청약일로부터 30일 이내에 승낙 또는 거절 통지를 해야 한다는 내용을 설명하고 있습니다. !! 회사는 계약의 청약을 받고 보험료 전액(보험료를 일시에 지급하기로 약정한 경우) 또는 제1회 보험  
료(보험료를 분납하기로 약정한 경우)(이하 ‘ 제1회 보험료 등’ 이라 합니다)를 받은 경우에는 청약일
부터 30일 이내에 승낙 또는 거절의 통지를 하며 통지가 없으면 승낙한 것으로 봅니다.


[Header Doc Index] 2 ------------------------
  Header 1:  제1조(보험계약의 성립)
    Header 2:  ③
      [Doc page_content]: 이 내용은 <제1조(보험계약의 성립)>의 일부로, 보험회사가 고객의 청약을 승낙하거나 거절할 경우 어떤 절차를 거치

# VectorDB 로 임베딩해서 저장

### 위에서 생성한 child chunk를 embedding해서 Chroma vector store에 저장

In [31]:
%%capture
!pip install chromadb

In [32]:
from langchain.vectorstores import Chroma

vector_store = Chroma.from_documents(documents=splits,
                                     embedding=BedrockEmbeddings()
                                    )

# Q&A Retrieval Chain

In [33]:
document_prompt = PromptTemplate(
        input_variables=["page_content"],
        template="Context:\ncontent:{page_content}\n",
    )

In [34]:
QUERY_PROMPT_TEMPLATE = """\
H:
Answer the question based on the provided context. Do not create false information. Answer only in the provided contexts.
{context}
Question: {question}
A:
"""

In [35]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain

In [36]:
QA_CHAIN_PROMPT = PromptTemplate.from_template(QUERY_PROMPT_TEMPLATE) # prompt_template defined above

In [37]:
llm_chain = LLMChain(llm=llm_text, prompt=QA_CHAIN_PROMPT, callbacks=None, verbose=True)

/tmp/ipykernel_14356/2127955414.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm_text, prompt=QA_CHAIN_PROMPT, callbacks=None, verbose=True)


In [38]:
combine_documents_chain = StuffDocumentsChain(
        llm_chain=llm_chain,
        document_variable_name="context",
        document_prompt=document_prompt,
        callbacks=None,
    )

/tmp/ipykernel_14356/3771212169.py:1: LangChainDeprecationWarning: This class is deprecated. Use the `create_stuff_documents_chain` constructor instead. See migration guide here: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain/
  combine_documents_chain = StuffDocumentsChain(


In [39]:
qa = RetrievalQA(
        combine_documents_chain=combine_documents_chain,
        callbacks=None,
        verbose=True,
        retriever=vector_store.as_retriever(search_kwargs={'k': 5}),
        return_source_documents=True,
    )

/tmp/ipykernel_14356/232904612.py:1: LangChainDeprecationWarning: This class is deprecated. Use the `create_retrieval_chain` constructor instead. See migration guide here: https://python.langchain.com/docs/versions/migrating_chains/retrieval_qa/
  qa = RetrievalQA(


# 질문에 대한 답변 테스트

In [40]:
question = '회사의 요구에 따르기 위하여 지출한 비용은 보상 받을 수 있나요?'

In [41]:
res = qa(question)

/tmp/ipykernel_14356/3271077376.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  res = qa(question)




> Entering new RetrievalQA chain...


> Entering new LLMChain chain...
Prompt after formatting:
H:
Answer the question based on the provided context. Do not create false information. Answer only in the provided contexts.
Context:
content:이 부분은 <document>의 제11조(보상하는 손해)에 해당합니다. 이 조항에서는 회사가 보장하는 위험으로 인해 발생한 손해에 대해 회사가 추가적으로 지급하는 비용에 대해 설명하고 있습니다. 이 부분은 잔존물 보전비용과 기타 협력비용에 대해 다루고 있습니다. !! 에 의해 회사가 잔존물을 취득한 경우에 한합니다.  
5. 기타 협력비용: 회사의 요구에 따르기 위하여 지출한 필요 또는 유익한 비용


Context:
content:이 청크는 문서의 <document>태그 내부에 있는 「제11조(보상하는 손해)」라는 헤더 아래에 속합니다. 이 헤더에서는 회사가 제1항에서 보장하는 위험으로 인해 발생한 손해에 대해 계약자 또는 피보험자가 지출한 특정 비용을 추가로 지급한다는 내용을 다루고 있습니다. !! 회사는 제1항에서 보장하는 위험으로 인하여 손해가 발생한 경우 계약자 또는 피보험자가 지출한 아  
래의 비용을 추가로 지급합니다.  
1. 잔존물 제거비용: 사고현장에서의 잔존물의 해체비용, 청소비용(사고현장 및 인근 지역의 토양, 대  
기 및 수질 오염물질 제거비용과 차에 실은 후 폐기물 처리비용은 포함되지 아니합니다) 및 차에
싣는 비용. 다만, 제1항에서 보장하지 아니하는 위험으로 보험의 목적이 손해를 입거나 관계법령에


Context:
content:이 문단은 「제11조(보상하는 손해)」 헤더에 속합니다. 회사는 특정한 위험으로 인해 발생한 손해에 대해 계약자 또는 피보험자가 지출한 비용을 추가로 지급하는데, 이 문단은 그 중 

In [42]:
print(res['result'])

네, 회사의 요구에 따르기 위하여 지출한 필요 또는 유익한 비용은 보상 받을 수 있습니다. 

제시된 컨텍스트에 따르면, 제11조(보상하는 손해)에서 다음과 같이 명시하고 있습니다:

"5. 기타 협력비용: 회사의 요구에 따르기 위하여 지출한 필요 또는 유익한 비용"

이는 회사가 보장하는 위험으로 인해 발생한 손해에 대해 회사가 추가적으로 지급하는 비용 중 하나로 포함되어 있습니다.
